# Task 1: Word Embeddings (10 points)

This notebook will guide you through all steps necessary to train a word2vec model (Detailed description in the PDF).

## Imports

This code block is reserved for your imports. 

You are free to use the following packages: 

(List of packages)

In [1]:
# Imports
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import torch as torch
from torch import nn
#nltk.download('stopwords')
#nltk.download("punkt")
from nltk.tokenize import word_tokenize
import math as mt
import time
import random
from joblib import Parallel, delayed
from torch.autograd import Variable

import torch.utils.data as data_utils
from torch.utils.data import DataLoader
print("Done")

Done


# 1.1 Get the data (0.5 points)

The Hindi portion HASOC corpus from [github.io](https://hasocfire.github.io/hasoc/2019/dataset.html) is already available in the repo, at data/hindi_hatespeech.tsv . Load it into a data structure of your choice. Then, split off a small part of the corpus as a development set (~100 data points).

If you are using Colab the first two lines will let you upload folders or files from your local file system.

In [2]:
#TODO: implement!

#from google.colab import files
#uploaded = files.upload()
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
# dev = 'cpu'
    
data = pd.read_csv('data/hindi_hatespeech.tsv', sep='\t')
data_development = data[:500]
# data_development = data
type(data_development['task_1'])

print("Done", dev)

Done cuda:0


## 1.2 Data preparation (0.5 + 0.5 points)

* Prepare the data by removing everything that does not contain information. 
User names (starting with '@') and punctuation symbols clearly do not convey information, but we also want to get rid of so-called [stopwords](https://en.wikipedia.org/wiki/Stop_word), i. e. words that have little to no semantic content (and, but, yes, the...). Hindi stopwords can be found [here](https://github.com/stopwords-iso/stopwords-hi/blob/master/stopwords-hi.txt) Then, standardize the spelling by lowercasing all words.
Do this for the development section of the corpus for now.

* What about hashtags (starting with '#') and emojis? Should they be removed too? Justify your answer in the report, and explain how you accounted for this in your implementation.

In [3]:
#TODO: implement!
hindi_stopword_file = open('data/stopwords-hi.txt', encoding="utf8")

sw_list = ['#', '?', '!', ';', ',', ':', "\'", '-', '=', '(', ')', '[', ']' , '{', '}', '"', '*', '@', '  ', '\\', '/', '..', '...', '....', '%'
          ,'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '\t']
sw_list_string = ''
for i in sw_list:
    sw_list_string+=i
hindi_stopwords = []
for x in hindi_stopword_file:
    hindi_stopwords.append(x.rstrip())

hindi_stopwords.extend(sw_list)
sentences = []
# text = data_development['text'].iloc[0] + ' @Zalman'
for text in data_development['text']:
    text_array = text.split(' ')
    # text_tokens =word_tokenize(text) #data_development["text"].apply(nltk.word_tokenize)
#     print(text_array)
    new_array = []
    for j in text_array:
        if '@' not in j and len(j) < 20:
            for char in sw_list:
                j = j.replace(char, '')
            new_array.append(j.lower())
    sentences.append(' '.join(new_array))
                
# tokens_without_sw = [word for word in text_tokens if not word in hindi_stopwords]
# filtered_sentence = (" ").join(tokens_without_sw)
# #text_tokens
# filtere_sentence
# print(sentences)

print("Done")

Done


## 1.3 Build the vocabulary (0.5 + 0.5 points)

The input to the first layer of word2vec is an one-hot encoding of the current word. The output od the model is then compared to a numeric class label of the words within the size of the skip-gram window. Now

* Compile a list of all words in the development section of your corpus and save it in a variable ```V```.

In [4]:
#TODO: implement!
temp_unique = [] # For unique words
temp_nounique = []
for j in sentences:
    temp2 = j.split(' ')
    for k in temp2:
        if k not in temp_unique:
            temp_unique.append(k)
        temp_nounique.append(k)
V = temp_unique
non_unique = temp_nounique
print(len(non_unique))
print(len(V))
print(sentences[0])

14395
4514
बांग्लादेश की शानदार वापसी भारत को  रन पर रोका indvban cwc


* Then, write a function ```word_to_one_hot``` that returns a one-hot encoding of an arbitrary word in the vocabulary. The size of the one-hot encoding should be ```len(v)```.

In [5]:
#TODO: implement!
def word_to_one_hot(word):
    if word in V:
        index = V.index(word)
        encoding = np.zeros(len(V))
        encoding[index]= 1
        return encoding.astype(np.uint8)
    return False
#   pass

## 1.4 Subsampling (0.5 points)

The probability to keep a word in a context is given by:

$P_{keep}(w_i) = \Big(\sqrt{\frac{z(w_i)}{0.001}}+1\Big) \cdot \frac{0.001}{z(w_i)}$

Where $z(w_i)$ is the relative frequency of the word $w_i$ in the corpus. Now,
* Calculate word frequencies
* Define a function ```sampling_prob``` that takes a word (string) as input and returns the probabiliy to **keep** the word in a context.

In [6]:
#TODO: implement!
def sampling_prob(word):
    z_wi = non_unique.count(word) / len(non_unique)
    try:
        p_wi = (mt.sqrt(z_wi / 0.001) + 1) * (0.001 / z_wi)
        return p_wi
    except ZeroDivisionError:
        print("Word doesn't exist in corpus")
#   pass

In [7]:
sampling_prob('बांग्लादेश')
sample_onehot = word_to_one_hot('बांग्लादेश')
print(sample_onehot.shape)
print(len(sentences))
print(sample_onehot)

(4514,)
500
[1 0 0 ... 0 0 0]


# 1.5 Skip-Grams (1 point)

Now that you have the vocabulary and one-hot encodings at hand, you can start to do the actual work. The skip gram model requires training data of the shape ```(current_word, context)```, with ```context``` being the words before and/or after ```current_word``` within ```window_size```. 

* Have closer look on the original paper. If you feel to understand how skip-gram works, implement a function ```get_target_context``` that takes a sentence as input and [yield](https://docs.python.org/3.9/reference/simple_stmts.html#the-yield-statement)s a ```(current_word, context)```.

* Use your ```sampling_prob``` function to drop words from contexts as you sample them. 

In [8]:
#TODO: implement!
def get_target_context(sentence, window_size):
    word = ''
    words_in_sentence = sentence.split(' ')
    for i in range(len(words_in_sentence)):
        context = []
        randd = random.random()
#         print('1st: ', words_in_sentence[i], sampling_prob(words_in_sentence[i]), randd)
        if randd <= sampling_prob(words_in_sentence[i]):
            word = words_in_sentence[i]
            upper_bound = i + window_size + 1
            lower_bound = i - window_size
            for j in range(lower_bound, upper_bound):
                rand2 = random.random()
                if i != j and j>=0 and j<len(words_in_sentence):
                    if rand2 <= sampling_prob(words_in_sentence[j]):
#                     print('2nd: ', words_in_sentence[i], words_in_sentence[j],sampling_prob(words_in_sentence[j]), rand2)
                        context.append(words_in_sentence[j])
        if len(word) > 0 and len(context) > 0:
            yield(word, context)
            
hello = get_target_context(sentences[22], 4)

print(sentences[22])
for val in hello:
    print(val)

you cry in front of ur god  of being deceived by the world    some fine day u die  reach hell    god tells you  you fucker or भोसडीके  no one deceived you more than urself. 
('you', ['cry', 'in', 'front', 'of'])
('cry', ['you', 'in', 'front', 'of', 'ur'])
('in', ['you', 'cry', 'front', 'of', 'ur', 'god'])
('front', ['you', 'cry', 'in', 'of', 'ur', 'god'])
('of', ['you', 'cry', 'in', 'front', 'ur', 'god', 'of'])
('ur', ['cry', 'in', 'front', 'of', 'god', '', 'of', 'being'])
('god', ['in', 'front', 'of', 'ur', 'of', 'being', 'deceived'])
('of', ['of', 'ur', 'god', '', 'being', 'deceived', 'by', 'the'])
('being', ['ur', 'god', 'of', 'deceived', 'by', 'the', 'world'])
('deceived', ['god', 'of', 'being', 'by', 'the', 'world'])
('by', ['', 'of', 'being', 'deceived', 'the', 'world'])
('the', ['of', 'being', 'deceived', 'by', 'world'])
('world', ['being', 'deceived', 'by', 'the', 'some'])
('some', ['world', 'fine', 'day', 'u', 'die'])
('fine', ['', 'some', 'day', 'u', 'die'])
('day', ['some', 

# 1.6 Hyperparameters (0.5 points)

According to the word2vec paper, what would be a good choice for the following hyperparameters? 

* Embedding dimension
* Window size

Initialize them in a dictionary or as independent variables in the code block below. 

In [9]:
# Set hyperparameters
# window_size = from 1 to 10
# window_size = from 500 to 1000 (640 used in paper)
window_size = 4
embedding_size = 600

# More hyperparameters
learning_rate = 0.05
epochs =500

# 1.7 Pytorch Module (0.5 + 0.5 + 0.5 points)

Pytorch provides a wrapper for your fancy and super-complex models: [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). The code block below contains a skeleton for such a wrapper. Now,

* Initialize the two weight matrices of word2vec as fields of the class.

* Override the ```forward``` method of this class. It should take a one-hot encoding as input, perform the matrix multiplications, and finally apply a log softmax on the output layer.

* Initialize the model and save its weights in a variable. The Pytorch documentation will tell you how to do that.

In [10]:
# Create model 

class Word2Vec(nn.Module):
    def __init__(self):
        super().__init__()
        initrange = 0.5 / embedding_size
#         self.w1 = Variable(torch.randn(len(V), embedding_size, device=dev).uniform_(-initrange, initrange).float(), requires_grad=True) # shape V*H
#         self.w2 = Variable(torch.randn(embedding_size, len(V), device=dev).uniform_(-initrange, initrange).float(), requires_grad=True)
        self.fc1 = nn.Linear(len(V), embedding_size)
        self.fc2 = nn.Linear(embedding_size, len(V))
# #         self.fc2 = nn.LogSoftmax()
# #         self.fc2 = nn.Lo
#         self.w1.weight.data.uniform_(-1,1)
#         self.w2.weight.data.uniform_(-1,1)
#         self.w1 = torch.nn.Parameter(torch.randn((len(V), embedding_size)), requires_grad = True)
#         self.w2 = torch.nn.Parameter(torch.randn((embedding_size, len(V))), requires_grad = True)


    def forward(self, one_hot):
#         hidden_layer = torch.matmul(one_hot, self.w1.type(torch.DoubleTensor))
#         matrix_mul = torch.matmul(hidden_layer, self.w2.type(torch.DoubleTensor))
#         e_x = torch.exp(matrix_mul - torch.max(matrix_mul))
#         log_softmax = torch.log(e_x / e_x.sum())
        x = self.fc1(one_hot.float())
# #         print(x.shape)
        x = self.fc2(x)
        log_softmax = torch.nn.functional.log_softmax(x, dim=1)
        return log_softmax
    
net = Word2Vec()
net.to(dev)
#     pass

print("Done")
w = []
for param in net.parameters():
    w.append(param.data)
# w = np.array(w)
# print(w[0].shape)

Done


# 1.8 Loss function and optimizer (0.5 points)

Initialize variables with [optimizer](https://pytorch.org/docs/stable/optim.html#module-torch.optim) and loss function. You can take what is used in the word2vec paper, but you can use alternative optimizers/loss functions if you explain your choice in the report.

In [11]:
# Define optimizer and loss
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)
criterion = nn.BCEWithLogitsLoss()

print("Done")

Done


In [12]:
for name, param in net.named_parameters():
    print(name, param.requires_grad)

fc1.weight True
fc1.bias True
fc2.weight True
fc2.bias True


In [13]:
def train():
    for e in range(epochs):
        for i in sentences:
            word_context = get_target_context(i, window_size)
            for word, context in word_context:
                input_vec = word_to_one_hot('reach')
                input_vec = input_vec.reshape((1,len(V)))
                input_vec = torch.tensor(input_vec)
                print(input_vec.requires_grad)
                for j in context:
                    output_vec = torch.tensor(word_to_one_hot('die')).reshape(1, len(V))
                    y_pred_torch = net(input_vec.to(dev))
                    loss = criterion(y_pred_torch, output_vec.to(dev))
                    print(loss.item())
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    time.sleep(2)

# def train2():
#     net = Word2Vec()
#     print("Training started")
#     for e in range(epochs):
#         count = 0
#         for i in sentences:
#             word_context = get_target_context(i, window_size)
#             for word, context in word_context:
#                 input_vec = word_to_one_hot('reach')
#                 input_vec = input_vec.reshape((1,len(V)))
#                 input_vec = torch.tensor(input_vec)
#                 input_vec.requires_grad = False
#                 for j in context:
# #                     output_vec = torch.tensor(word_to_one_hot(j)).reshape(1, len(V))
#                     output_vec = torch.tensor(word_to_one_hot('die')).reshape(1, len(V))
#                     y_pred_torch = net(input_vec)
#                     loss = criterion(y_pred_torch, output_vec.to(dev))
#                     print(loss.item())
#                     optimizer.zero_grad()
#                     loss.backward()
#                     optimizer.step()
#                     time.sleep(2)
train()

print("Training finished")

False


RuntimeError: result type Float can't be cast to the desired output type Byte

In [13]:
def create_dataset(sentences):
    x_train = []
    y_train = []
    print("started")
    for j, i in enumerate(sentences):
#         if count % 200 == 0:
#             print((j+1)*count)
#             count = 0
        if j % 50 == 0:
#         if len(x_train) > 0:
        
#             print(type(x_train[0][0][0]))
            print("INDEX:", j, "Length:",len(x_train))
        word_context = get_target_context(i, window_size)
        for word, context in word_context:
            input_vec = word_to_one_hot(word)
            input_vec = input_vec.reshape((1,len(V)))
            for j in context:
                output_vec = word_to_one_hot(j).reshape(1, len(V))
                x_train.append(input_vec)
                y_train.append(output_vec)
                
    x_train = np.array(x_train).reshape(len(x_train), len(V))
    y_train = np.array(y_train).reshape(len(x_train), len(V))
    return x_train, y_train
print(len(sentences))
x_train, y_train = create_dataset(sentences)
print(x_train.shape)
# Parallel(n_jobs=-1)(delayed(create_dataset)(d) for d in sentences)

500
started
INDEX: 0 Length: 0
INDEX: 50 Length: 6165
INDEX: 100 Length: 12112
INDEX: 150 Length: 18383
INDEX: 200 Length: 24311
INDEX: 250 Length: 29692
INDEX: 300 Length: 36622
INDEX: 350 Length: 43813
INDEX: 400 Length: 49553
INDEX: 450 Length: 55184
(61664, 4514)


In [14]:
print(y_train[0])
batch_size = 200
x_train = torch.from_numpy(x_train)
y_train = torch.Tensor(y_train)
x_train = data_utils.TensorDataset(x_train, y_train)
train_dl = DataLoader(x_train, batch_size=batch_size, shuffle=True)
# del x_train_tensor
del x_train
print(len(y_train))

[0 0 1 ... 0 0 0]
61664


# 1.9 Training the model (3 points)

As everything is prepared, implement a training loop that performs several passes of the data set through the model. You are free to do this as you please, but your code should:

* Load the weights saved in 1.6 at the start of every execution of the code block
* Print the accumulated loss at least after every epoch (the accumulate loss should be reset after every epoch)
* Define a criterion for the training procedure to terminate if a certain loss value is reached. You can find the threshold by observing the loss for the development set.

You can play around with the number of epochs and the learning rate.

In [15]:
print(y_train.shape)

torch.Size([61664, 4514])


In [44]:

# Define train procedure

# load initial weights
# count = 0
def train():
    for e in range(epochs):
        loss_per_epoch = 0
        for i in sentences:
            word_context = get_target_context(i, window_size)
            for word, context in word_context:
                input_vec = word_to_one_hot(word)
                input_vec = input_vec.reshape((1,len(V)))
                input_vec = torch.tensor(input_vec)
#                 print(input_vec.requires_grad)
                # input_vec.requires_grad  = False
                for j in context:
                    output_vec = torch.tensor(word_to_one_hot(j), dtype=torch.float64).reshape(1, len(V))
                    y_pred_torch = net(input_vec.to(dev))
                    loss = criterion(y_pred_torch, output_vec.to(dev))
#                     print(loss.item())
                    loss_per_epoch += loss.item()
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
#                     time.sleep(2)
        
        print('Epoch:',e,"\tLoss:",loss_per_epoch / y_train.shape[0])

train()

print("Training finished")

KeyboardInterrupt: 

In [16]:

# Define train procedure

# load initial weights
# count = 0
def train():
    print("Started:")
    for e in range(epochs):
        loss_per_epoch = 0
        for i, (X_batch, y_batch) in enumerate(train_dl):
            X_batch, y_batch = X_batch.to(dev), y_batch.to(dev)
            input_vec = word_to_one_hot(X_batch)
            input_vec = input_vec
            input_vec = torch.tensor(input_vec)
#                 print(input_vec.requires_grad)
            # input_vec.requires_grad  = False
#             for j in context:
            output_vec = torch.tensor(y_batch, dtype=torch.float64)
            y_pred_torch = net(X_batch)
            loss = criterion(y_pred_torch, y_batch)
            loss_per_epoch += loss.item()
#                     print(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
#                     time.sleep(2)
        
        print('Epoch:',e,"\tLoss:",loss_per_epoch / y_train.shape[0])

train()

print("Training finished")

Started:


<ipython-input-16-b152f7e25b57>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_vec = torch.tensor(y_batch, dtype=torch.float64)


Epoch: 0 	Loss: 1.0450070554856452e-05
Epoch: 1 	Loss: 1.0450067281241896e-05
Epoch: 2 	Loss: 1.0450072865643198e-05
Epoch: 3 	Loss: 1.0450080911863062e-05
Epoch: 4 	Loss: 1.045006973928466e-05
Epoch: 5 	Loss: 1.0450083475628096e-05
Epoch: 6 	Loss: 1.045008134985532e-05
Epoch: 7 	Loss: 1.0450082146548137e-05
Epoch: 8 	Loss: 1.0450061696840592e-05
Epoch: 9 	Loss: 1.0450076883089437e-05
Epoch: 10 	Loss: 1.0450077358839649e-05
Epoch: 11 	Loss: 1.0450080368148532e-05
Epoch: 12 	Loss: 1.0450073035553989e-05
Epoch: 13 	Loss: 1.0450073866228963e-05
Epoch: 14 	Loss: 1.0450074466580422e-05
Epoch: 15 	Loss: 1.0450071812196299e-05
Epoch: 16 	Loss: 1.0450067530444388e-05
Epoch: 17 	Loss: 1.0450059782512357e-05
Epoch: 18 	Loss: 1.04500722162064e-05
Epoch: 19 	Loss: 1.0450077917657359e-05
Epoch: 20 	Loss: 1.0450090660966622e-05
Epoch: 21 	Loss: 1.0450087153252754e-05
Epoch: 22 	Loss: 1.0450088810826906e-05
Epoch: 23 	Loss: 1.045008633768096e-05
Epoch: 24 	Loss: 1.0450057898390484e-05
Epoch: 25 	Loss

RuntimeError: CUDA error: unspecified launch failure

# 1.10 Train on the full dataset (0.5 points)

Now, go back to 1.1 and remove the restriction on the number of sentences in your corpus. Then, reexecute code blocks 1.2, 1.3 and 1.6 (or those relevant if you created additional ones). 

* Then, retrain your model on the complete dataset.

* Now, the input weights of the model contain the desired word embeddings! Save them together with the corresponding vocabulary items (Pytorch provides a nice [functionality](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for this).